In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = "../../../../data/testing/train"

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode="nearest",
    validation_split=0.2
)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    subset="validation"
)

Found 2777 images belonging to 7 classes.
Found 690 images belonging to 7 classes.


In [ ]:
test_dir = "../../../../data/testing/test"
test_datagen = ImageDataGenerator(
    rescale=1.0/255.0   
)


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical",
    shuffle=False
)

In [ ]:
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu")
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

AttributeError: module 'keras._tf_keras.keras.layers' has no attribute 'AdaptiveAveragePooling2D'

In [12]:

train_generator.samples // 32

86

In [7]:
history = model.fit(
    train_generator,
    validation_data = val_generator,
    epochs=2,
    steps_per_epoch=train_generator.samples,
    validation_steps=val_generator.samples
)

Epoch 1/2
2777/2777 ━━━━━━━━━━━━━━━━━━━━ 124s 44ms/step - accuracy: 0.3015 - loss: 1.7615 - val_accuracy: 0.3290 - val_loss: 1.6924
Epoch 2/2
2777/2777 ━━━━━━━━━━━━━━━━━━━━ 143s 51ms/step - accuracy: 0.3116 - loss: 1.6968 - val_accuracy: 0.3333 - val_loss: 1.6091


Found 637 images belonging to 7 classes.


In [10]:
testloss, testaccuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {testaccuracy:.4f}")

/Users/devadattamandaogane/NCSU_Subjects/Fall_2024/GeospatialAI/rsfedml/fedenv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


20/20 ━━━━━━━━━━━━━━━━━━━━ 19s 959ms/step - accuracy: 0.3315 - loss: 1.5575
Test Accuracy: 0.2983
